# 前端实战私房手册

## CSS

### `:nth-first-child(Xn+Y)`的n是从0还是从1开始

如果是偶数，比如`2n`，`3n`，则从1开始，因此选择的是集合中的`2, 4, 6...`或者`3,6,9...`。如果是奇数，比如`2n+1`,`3n+1`，则从0开始，因此选中的是集合中的`1，3，5...`，或者`1, 4, 7...`

## EasyUI

### 如何判断一个datagrid是否已经创建以及销毁datagrid

官方文档里面没有写，可以使用如下的代码判断是否创建一个`datagrid`:
```javascript
$('#data-grid').data('datagrid')
```
如果要销毁一个已经存在的`datagrid`，可以这样：
```javascript
const p = $('#data-grid').datagrid('getPanel')
p.panel('destroy')
```
注意，不能通过`$('#data-grid').datagrid('getPanel')`来判断`datagrid`是否存在，如果页面还没有创建一个`datagrid`，此时会报错。

## javascript

### 文件上传和下载

#### ajax文件上传

- [Ajax上传文件及携带参数](https://www.cnblogs.com/huiy/p/12833307.html)
- [FormData的使用及请求头分析](https://www.orzzone.com/formdata-usage-request-header-analysis.html)
- [FormData和Content-Type: multipart/form-data](https://www.cnblogs.com/nicholaswang/p/11459860.html)
- [JavaScript FormData的详细介绍及使用](https://blog.csdn.net/liupeifeng3514/article/details/78988001)

使用ajax上传文件要注意，ajax本身不支持文件上传，因此实际上是利用`FormData`对象，构造一个表单进行上传，例子如下：
```javascript
$("#update-btn").click(function (e) {
    const uploadForm = new FormData($("#upload-form")[0])
    $.ajax({
        url: `http://127.0.0.1:8000/upload/${rftype}/`,
        type: "POST",
        cache: false,
        data: uploadForm,
        processData: false, # 告知jquery不要处理数据
        contentType: false, # 告知jquery不要添加表头，使用FormData构造的表单会自动为multipart表头
        success: function (res) {
            showMsg("提示", res, 0)
        },
        error: function (jqXHR, textStatus, errorThrown) {
            showMsg("批量更新添加", "添加失败，请检查文件内容是否正确", 0)
        }
    });
})
```
注意注释的部分。

#### fetch异步下载数据

- [fetch MSN官方文档说明](https://developer.mozilla.org/zh-CN/docs/Web/API/Fetch_API/Using_Fetch)

一般情况下，普通的文件通过a链接，添加download的属性就可以了，此时会作为附件下载而不会选择使用浏览器打开。但是这种方法是同步的，如果要实现异步下载，可以使用`fetch`，注意，传统的ajax貌似很难完美实现异步下载，虽然可以通过ajax发送请求，但是返回的文件数据会保存在javascript的内存空间，无法下载到本地。可以借用一些技巧，比如使用javascript模拟表单进行下载，但是使用`fetch`更加方便和优雅。
基本的思路就是调用`fetch`返回的`response`对象的`blob`方法，将返回的数据包装为一个`blob`对象，然后使用`URL.createObjectURL`为`blob`对象创建一个链接，在代码中临时创建一个`a`元素，其`url`地址指向`blob`对象，从而实现下载的目的。

代码如下：
```javascript
function downloadFile(fileUrl) {
    $.messager.progress({
        title: `正在下载，请稍后...`
    })
    fetch(fileUrl)
        .then(resp => {
            const filename = getFilename(resp)
            return Promise.all([filename, resp.blob()]) // 注意：resp.blob()返回的是一个Promise
        })
        .then(([filename, blob]) => {
            const url = window.URL.createObjectURL(blob)
            const a = document.createElement('a')
            a.style.display = 'none'
            a.href = url
            a.download = filename
            document.body.appendChild(a)
            a.click()
            window.URL.revokeObjectURL(url)
            document.body.removeChild(a)
            $.messager.progress('close')
        })
        .catch((error) => {
            $.messager.progress('close')
            $.messager.alert("出错了", error)
        })
}
``` 

有几点要注意，如下：
1. `resp.blob()`返回的是一个promise，resolve以后才会返回一个blob对象，因此，如果还有其它参数要传递的话，需要使用`Promise.all`，比如上面的例子，如果直接`return [filename, resp.blob()]`，由于列表是普通的对象，因此会立刻`resolve`，此时后面的`then`接受到的不是一个`blob`对象，而是一个`pending`的`Promise`。
2. 通过`createObjectURL`和`createElement`创建临时的`url`链接和`a`元素，文件下载以后记得分别调用`revokeObjectURL`和`removeChild`进行释放。

### ajax实现PUT和DELETE请求

- [jquery的ajax实现PUT与DELETE请求](https://blog.csdn.net/weixin_41556165/article/details/82414822)